In [34]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer() 

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    return res

In [35]:
import pandas as pd

df = pd.read_csv('res_mul_all.csv')
df.head()

,id,reviewID,sentenceID,review,category,polarity,entity,preprocessed_sentence,type_sentence
0,0,RL#3,RL#3:1,I am not necessarily fanatical about this plac...,VALUE#PRICES,positive,VALUE,i am not necessarily fanatical about this plac...,compound_sentence
1,2,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
2,3,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
3,4,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
4,5,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence


In [36]:
text_data = []
for sentence in df['review']:
    tokens = preprocess(sentence)
    text_data.append(tokens)

In [37]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [38]:
import gensim

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 30, id2word=dictionary, random_state = 42)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics()

In [39]:
for a in topics:
    print(a)
    print()

(11, '0.054*"food" + 0.027*"high" + 0.027*"price" + 0.018*"pay" + 0.018*"get" + 0.018*"always" + 0.018*"really" + 0.018*"going" + 0.018*"terrible" + 0.018*"portion"')

(29, '0.030*"relaxed" + 0.024*"friendly" + 0.024*"atmosphere" + 0.024*"service" + 0.024*"casual" + 0.018*"fast" + 0.018*"great" + 0.012*"new" + 0.012*"duck" + 0.012*"world"')

(7, '0.027*"place" + 0.020*"food" + 0.014*"fish" + 0.014*"price" + 0.014*"cream" + 0.014*"really" + 0.014*"reasonably" + 0.014*"like" + 0.014*"mahi" + 0.014*"flavorful"')

(1, '0.028*"food" + 0.019*"best" + 0.017*"try" + 0.017*"service" + 0.011*"pleasant" + 0.011*"bad" + 0.011*"decor" + 0.011*"atmosphere" + 0.011*"nice" + 0.011*"staff"')

(13, '0.068*"food" + 0.049*"price" + 0.032*"high" + 0.023*"bad" + 0.019*"staff" + 0.014*"bagel" + 0.014*"great" + 0.009*"going" + 0.009*"always" + 0.009*"way"')

(2, '0.034*"like" + 0.020*"food" + 0.014*"cool" + 0.014*"sea" + 0.014*"better" + 0.014*"sushi" + 0.014*"served" + 0.014*"taste" + 0.014*"would" + 0.009*"

In [40]:
all_topics = []
for topic in topics:
    for pair in topic[1].split(' + '):
        word = pair.split('*')[1].replace("\"", '')
        all_topics.append(word)

In [41]:
import pandas as pd

def aspect_topic(all_topics):
    sf = pd.DataFrame(columns=['id','review','category','term'])
    count = 0
    index = 0
    res = []
    for sentence in df['review']:
        lowercased = sentence.lower()
        term = []
        category = []
        for cat in df['category'][index].split(','):
            category.append(cat.split('#')[0])
        id_name = df['id'][index]
        for topic in all_topics:
            if topic in lowercased:
                term.append(topic)
#         print(term)
        if len(term) == 0:
            print(sentence)
            count += 1
        sf = sf.append({'id': id_name, 'review': sentence.strip().lower().replace('  ', ' '), 'category': '|'.join(category), 'term': '|'.join(term)}, ignore_index=True)
        index += 1
    print(count)
    sf.to_csv("lda.csv")
    sf.to_excel("lda.xlsx")

In [42]:
aspect_topic(list(dict.fromkeys(all_topics)))

0


In [9]:
# list(dict.fromkeys(all_topics))